<a href="https://colab.research.google.com/github/jalyngearries/GRCATimeSeries/blob/main/GRCA_Time_Series_Spring_2024_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Planet Imagery Earth Engine Delivery

In [ ]:
## Import the Planet API
import planet
from planet import Session, DataClient, OrdersClient

In [ ]:
## Import the Earth Engine API
import ee

## Importing other packages for ordering and delivery
import json
import os
import pathlib
import time
import shapely
import requests
import geopandas as gpd
from google.colab import drive
import asyncio

### Step 1: Authenticate Earth Engine in Python environment

In [ ]:
## Trigger authentication flow
ee.Authenticate()

## Initialize the library
ee.Initialize(project='ee-jg3648')

## (Optional) Mount Google Drive to Colab environment (where my AOIs are stored)
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Step 2: Authenticate Planet account for Orders API

In [ ]:
## Set up & authenticate session
from getpass import getpass
from planet import Auth
api_key = getpass('Enter your API key:')

auth = Auth.from_key(api_key)

Enter your API key:··········


### Step 3: Define cloud delivery location for images

In [ ]:
## Define a cloud delivery configuration object (defining the destination to be GEE)
cloud_config = planet.order_request.google_earth_engine(
    project='ee-jg3648', collection='C122_collection')

## Define delivery configuration
delivery_config = planet.order_request.delivery(cloud_config=cloud_config)

### Step 4: Define AOI to clip images

In [ ]:
with open("/content/gdrive/MyDrive/NAU - PhD INF/Sites/122.8 - 122 Mile Canyon AOI.geojson") as f:
  C122_json = json.load(f)

C122_coordinates = C122_json['features'][0]['geometry']['coordinates'][0]

print(C122_coordinates)


[[-112.52124115899994, 36.249611894000054], [-112.52113909699995, 36.23635840700007], [-112.47037120499999, 36.236604132000025], [-112.47046469599996, 36.24985773700007], [-112.52124115899994, 36.249611894000054]]


In [ ]:
C122_AOI = {
    "type":"Polygon",
    "coordinates": [[[-112.52124115899994, 36.249611894000054],
                     [-112.52113909699995, 36.23635840700007],
                     [-112.47037120499999, 36.236604132000025],
                     [-112.47046469599996, 36.24985773700007],
                     [-112.52124115899994, 36.249611894000054]]]
}

### Step 5: Define images to order

In [ ]:
C122_Images = ('20210217_173622_00_2264',
                '20210317_173744_48_2233',
                '20210416_172946_90_2457',
                '20210515_173753_36_2259',
                '20210616_172509_84_2449',
                '20210711_181710_01_2414',
                '20210815_181652_51_2403',
                '20210916_173642_82_2262',
                '20211015_172756_18_2439',
                '20220216_172108_19_2457',
                '20220317_172119_88_2442',
                '20220415_174952_87_2486',
                '20220514_172150_86_2465',
                '20220615_171820_30_2453',
                '20220716_171716_96_2429',
                '20220816_171436_07_2463',
                '20220918_175041_55_2438',
                '20221017_175032_95_2486',
                '20230216_173936_92_2251',
                '20230316_171706_04_241f',
                '20230417_175654_73_249e',
                '20230512_180734_09_2414',
                '20230614_171801_36_242b',
                '20230715_172220_76_24af',
                '20230825_165725_96_2445',
                '20230915_180134_51_248e',
                '20231015_172436_91_2442',
                '20240216_173323_32_2439',
                '20240314_173535_82_242d'
)
image_ids = [image_id for image_id in C122_Images]

### Step 6: Build your order request

In [ ]:
from planet import order_request

item_ids = C122_Images

products = [
    order_request.product(item_ids, 'analytic_udm2', 'PSScene')
]

tools = [
    order_request.reproject_tool(projection='EPSG:4326', kernel='cubic'),
    order_request.clip_tool(C122_AOI)
]
C122_request = order_request.build_request(
    '122 Images', products=products, tools=tools, delivery=delivery_config,
)

### Step 7: Create order request

In [ ]:
from planet import Session, OrdersClient

# an async Orders client to request order creation
async def main():
  async with Session(auth=auth) as sess:
    cl = OrdersClient(sess)
    order = await cl.create_order(C122_request)

# async magic to remember: "async def" to create a coroutine, then "await" to make it run
await main()